# Practical session demo (students)

We are going to take a look at the different functionalities:
* Create a design of experiments
* Sample from this space
* Evaluate the samples using a well-known benchmark function
* Iteratively find the minimum of the loss-landscape using a global optimizer
* Look at one of the example experiments: 20D Ackley optimization with CMAES

First, we download the solution to the exercises from GitHub in order to check your answers:

In [1]:
!wget -q -cO - https://github.com/mpvanderschelling/F3DASM_practical/blob/main/session1/exercise_best_sample.obj?raw=true > exercise_best_sample.obj
!wget -q -cO - https://github.com/mpvanderschelling/F3DASM_practical/blob/main/session1/exercise_adam_optimization.obj?raw=true > exercise_adam_optimization.obj
!wget -q -cO - https://github.com/mpvanderschelling/F3DASM_practical/blob/main/session1/exercise_pso_optimization.obj?raw=true > exercise_pso_optimization.obj
!wget -q -cO - https://github.com/mpvanderschelling/F3DASM_practical/blob/main/session1/exercise_samples_lhs.obj?raw=true > exercise_samples_lhs.obj
!wget -q -cO - https://github.com/mpvanderschelling/F3DASM_practical/blob/main/session1/exercise_samples_random.obj?raw=true > exercise_samples_random.obj

import the `f3dasm` package:

In [2]:
# If f3dasm is not found, install the correct version from pip
try:
    import f3dasm
except ModuleNotFoundError:
    %pip install f3dasm==0.2.5 --quiet
    import f3dasm

Note: you may need to restart the kernel to use updated packages.


2022-11-30 11:53:30.154556: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 11:53:30.311063: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-30 11:53:30.967910: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64
2022-11-30 11:53:30.967983: W tensorflow/compiler/xla/stream

In [3]:
# Check if you got the right version, otherwise uninstall this version!

import os

if f3dasm.__version__ != '0.2.5':
    %pip uninstall -y f3dasm
    os._exit(00)
    # The kernel will be restarted and you need to run the notebook again!
    
print(f"Your f3dasm version is {f3dasm.__version__}!")

Your f3dasm version is 0.2.5!


import some other packages:

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import pytest
from cycler import cycler

make sure that we specify a `seed`:

In [5]:
seed = 42

### Design: Creating the design space

**Parameters**

There are three different parameters you can construct:
* We can create continuous parameters with a `lower_bound` and `upper_bound` (optional) with the `f3dasm.ContinuousParameter` class
* We can create discrete parameters with a `lower_bound` and `upper_bound` with the `f3dasm.DiscreteParameter` class
* We can create categorical parameters with a list of strings (categories) with the `f3dasm.CategoricalParameter` class

all parameters also require a `name`.

**Design space**

Parameters define the `f3dasm.DesignSpace` and can be constructed by calling the `f3dasm.DesignSpace` class and providing:
* a list of input parameters (`input_space`) 
* a list of output parameters (`output_space`)


***
**Exercise #1**

Create a `DesignSpace` with the following features:
* 2 continuous input parameters named `x0` and `x1`, both with the same range of `-1.0` to `1.0`
* 1 continuous output parameter named `y`
* Call this designspace `design`
***

In [6]:
## YOUR CODE HERE ##

In [19]:
## CHECK ##

assert design == f3dasm.make_nd_continuous_design(bounds=np.tile([-1., 1.], (2,1)), dimensionality=2), "Your answer is incorrect :("
print("Correct answer!")

Correct answer!


What's next? We can sample from this design space with the **sampling** block!

### Sampling: Latin Hypercube Sampler

Samplers can be found within the submodule `f3dasm.sampling`. Currently 3 samplers are readily available:
* Random Uniform Sampling (`f3dasm.sampling.RandomUniform`)
* [Latin Hypercube Sampling](https://en.wikipedia.org/wiki/Latin_hypercube_sampling)(`f3dasm.sampling.LatinHypercube`)
* [Sobol Sequence Sampling](https://en.wikipedia.org/wiki/Sobol_sequence) (`f3dasm.sampling.SobolSequence`)

**Creating a sampler object**
A new sampler can be created by initializing the sampler with:
* A design space
* A random seed (optional)

After that, you can sample by invoking the `get_samples(numsamples)`, where `numsamples` is the number of samples you want to get

***
<b>Exercise #2</b>

Now you are going to create a sampler and sample some points from the designspace:
* Create a <u>Random Uniform sampling</u> with the designspace and seed described above
* Sample 30 points from the designspace
* Store the resulting `Data`-object in the variable `samples_random`
* Do the same with <u>Latin Hypercube sampling</u> and store the samples as `samples_lhs`
***

In [9]:
## YOUR CODE HERE ##



# If you are stuck you may uncomment the following lines to import the data object directly:
# samples_lhs = f3dasm.read_pickle('exercise_samples_lhs')
# samples_random = f3dasm.read_pickle('exercise_samples_random')

In [23]:
## CHECK ##

assert samples_lhs.data.equals(f3dasm.read_pickle('exercise_samples_lhs').data), "Your answer is incorrect :("
assert samples_random.data.equals(f3dasm.read_pickle('exercise_samples_random').data), "Your answer is incorrect :("
print("Correct answer!")

EOFError: Ran out of input

We can plot the datapoints with the `data.plot()` function:

In [ ]:
fig, ax = samples_lhs.plot(input_par1='x0', input_par2='x1')

In [ ]:
samples_random.data

As you could see earlier, the output values are all `NaN`. Let's evaluate the samples with a benchmark function from the **simulation** block!

### Simulation: Evaluating a benchmark function

Several benchmark functions have been implemented in the submodule `f3dasm.functions` to work with **continuous and single-objective** optimization problems.

These functions require an input-vector $\mathbf{x}$ and output a scalar $f(\mathbf{x})$

The `Levy()` function is a well-known, multimodal function:

$f(\textbf{x}) = \sin^2 (\pi w_1) + \sum_{i = 1}^{D - 1} (w_i - 1)^2 \left( 1 + 10 \sin^2 (\pi w_i + 1) \right) + (w_d - 1)^2 (1 + \sin^2 (2 \pi w_d)) \\ w_i = 1 + \frac{x_i - 1}{4}`
$


First we create such a function `f` by creating an object from the `f3dasm.functions.Levy` class

> *The original input domain is (-10, 10), but we scale these boundaries to the boundaries our input space*

In [ ]:
dim = 2
domain = np.tile([-1., 1.], (dim,1))

In [ ]:
f = f3dasm.functions.Levy(dimensionality=dim, seed=seed, scale_bounds=design.get_bounds())

The global minima are known for these functions and can be accesed by the `get_global_minimum()` function:

In [ ]:
x_min, y_min = f.get_global_minimum(dim)
print(f"The global minimum is {y_min} at position {x_min}")

We can plot a three-dimensional loss-landscape of the two input parameters $x_0$ and $x_1$ with the `plot()` function:
* `px` denotes the resolution on each axis
* `domain` denotes the domain of the two axis to be plotted

In [ ]:
fig, ax = f.plot(px=100, domain=domain)

Evaluating the function is easy, just pass either:
* the `Data()` object to it: `get_input_data()`
* or a 2D numpy array: `get_input_data().to_numpy()`

.. and all the samples will be evaluated.
The output will be a `np.ndarray`

In [ ]:
x = samples_lhs.get_input_data()

By calling the `add_output` option, we can add individual columns or overwrite data to our DataFrame:

In [ ]:
samples_lhs.add_output(output=f(x.to_numpy()))

In [ ]:
samples_lhs.show()

We can create a contour plot with the samples coordinates with the `plot_data()` function: 

In [ ]:
fig, ax = f.plot_data(samples_lhs,px=300,domain=domain)

> The red star will indicate the best sample



***
**Exercise #3**

* By consulting the documentation, find the location of the best sample obtainted by Latin Hypercube sampling (the red star)
* Store this value as `x_best`
***

In [ ]:
## YOUR CODE HERE ##


In [ ]:
## CHECK ##

assert (x_best == f3dasm.read_pickle('exercise_best_sample')).all(), "Your answer is incorrect :("
print("Correct answer!")

Now we will take a look how we can find the minimum of this loss-function with an optimizer!

### Optimization: CMAES optimizer

Optimizers can be found in the submodule `f3dasm.optimization` and are ported from several libraries such as `GPyOpt`, `scipy-optimize`, `tensorflow` and `pygmo`

We will use the CMAES optimizer to find the minimum. We can find an implementation in the `f3dasm.optimization` module:

In [ ]:
optimizer = f3dasm.optimization.CMAES(data=copy.copy(samples_lhs), seed=seed)

By calling the `iterate()` method and specifying the function and the number of iterations, we will start the optimization process:

In [ ]:
optimizer.iterate(iterations=100, function=f)

After that, we can extract the data:

In [ ]:
cmaes_data = optimizer.extract_data()

... and create a contour plot again:

In [ ]:
fig, ax = f.plot_data(data=cmaes_data,px=300, domain=domain)

**Hyperparameters**

You can overwrite the default hyperparameters of each optimizer by supplying a dictionary to the optimizer initializer.
The parameters of the optimizer can be viewed by calling the `parameter` attribute:

In [ ]:
optimizer.parameter

***
**Exercise #4**
* Retrieve the default `learning_rate` parameter of the `Adam` optimizer
* Optimize the same `Levy()`-function 50 iterations but now with the `Adam` optimizer with a `learning_rate` parameter of `2.0e-3`
* Extract the data and call it `adam_data`
* Plot the loss-landscape and data with the `plot_data` function
***


In [ ]:
## YOUR CODE HERE ##


In [ ]:
## CHECK ##

assert f3dasm.read_pickle('exercise_adam_optimization').data.to_numpy() == pytest.approx(adam_data.data.to_numpy()), "Your answer is incorrect :("
print("Correct answer!")

### Experiment: Multiple realizations of SGD on 20D Ackley function

Now we take a look at an example of an experiment where use the following blocks to optimize a 20D Ackley function with the CMAES optimizer over 10 different realizations:

In [ ]:
# Define the blocks:
dimensionality = 20
iterations = 600
realizations = 10

hyperparameters= {} # If none are selected, the default ones are used

domain = np.tile([-1., 1.], (dimensionality,1))
design = f3dasm.make_nd_continuous_design(bounds=domain, dimensionality=dimensionality)
data = f3dasm.Data(design)

In [ ]:
# We can put them in a dictionary if we want

implementation = {
'realizations': realizations,
'optimizer': f3dasm.optimization.SGD(data=data, hyperparameters=hyperparameters), 
'function': f3dasm.functions.Ackley(dimensionality=dimensionality, scale_bounds=design.get_bounds()),
'sampler': f3dasm.sampling.LatinHypercube(design, seed=seed),
'iterations': iterations,
'seed': seed
}

The `run_multiple_realizations()` function will be the pipeline of this experiment:

In [ ]:
results = f3dasm.run_multiple_realizations(**implementation)

In [ ]:
cc = (cycler(color=list('bgrcmyk')) *
      cycler(linestyle=['-', '--', 'dotted']))

In [ ]:
def calc_mean_std(results):
    mean_y = pd.concat([d.get_output_data().cummin() for d in results], axis=1).mean(axis=1)
    std_y = pd.concat([d.get_output_data().cummin() for d in results], axis=1).std(axis=1)
    return mean_y, std_y

In [ ]:
def create_axis(results: f3dasm.OptimizationResult, ax):
    mean_y, _ = calc_mean_std(results.data)

    ax.plot(mean_y, label=f"optimizer={results.optimizer}")
    return ax

In [ ]:
def plot_results(results: f3dasm.OptimizationResult, logscale: bool = True, ax = None, fig = None):
    if fig is None:
        fig = plt.figure(figsize=(15,6))
    
    if ax is None:
        ax = plt.axes()

    ax.set_xlabel('iterations')
    ax.set_ylabel('f(x)')
    if logscale:
        ax.set_yscale('log')
    ax.set_prop_cycle(cc)

    for _, res in enumerate([results]):
        ax = create_axis(res, ax)

    ax.legend(loc='upper right', ncol=3, fancybox=True, shadow=True)
    return fig, ax

In [ ]:
fig, ax = plot_results(results, logscale=False)

We can change a hyper-parameter such as the `learning_rate` and rerun the experiment with ease:

In [ ]:
implementation.update({'optimizer': f3dasm.optimization.SGD(data=data, hyperparameters={'learning_rate': 3e-2})})
results_2 = f3dasm.run_multiple_realizations(**implementation)

In [ ]:
fig2, ax2 = plot_results(results_2, logscale=False, ax=ax, fig=fig)
fig2

***

**Exercise #5**

Replicate the following experiment:
* Create a 6-dimensional, continuous design space with bounds for every dimension `-1.0` and `1.0`.
* Create a 6-dimensional, noiseless `Schwefel` function
* Create 40 initial samples by sampling from `SobolSequence`
* Optimize the function for 500 iterations and 5 realizations with the `PSO` optimizer. Use the default hyperparameters.
* Store the resulted `f3dasm.OptimizerResult`-object in the variable `pso_data`

*Hint #1: you can use the `f3dasm.make_nd_continuous_design()` function to quickly make a suitable design space.*

*Hint #2: a suitable domain has already been coded for you*
****

In [ ]:
## YOUR CODE HERE ##

# Use this domain
dimensionality = 6
domain = np.tile([-1., 1.], (dimensionality,1))

In [ ]:
## CHECK ##

pso_data: f3dasm.OptimizationResult = f3dasm.read_pickle('exercise_pso_optimization')

for index, result in enumerate(pso_data.data):
    assert result.data.to_numpy() == pytest.approx(pso_data.data[index].data.to_numpy()), "Your answer is incorrect :("

print("Correct answer!")

The average performance of the optimize can be plotted:

In [ ]:
plot_results(pso_data, logscale=False)

This marks the end of the first practical session of the `f3dasm` framework!

If you have any comments, questions or remarks feel free to reach out to me!

***